# LLM Optimization Modelling Experiment

In [3]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [55]:
problem = '''You are a chemist in charge of a process that requires two liquids to reach the end product. Your goal is to maximize the quality of the product in the end, which is measured by the difference between the Napierian logarithm of the product of the litres used of each component and the cube root of the summation of the first component litres as well as double the second component litres. It is required that the squared difference between the litres employed of each component and the mean litres utilised of both components is below ten. This is necessary for the obtained set to be stable. Furthermore, the mixture tank available to you can only fit 80 l. Please formulate a mathematical optimization model for this problem. '''

## 2. Generate the mathematical model

In [56]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [57]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model 

**Parameters:**

*  None specified in the problem statement. 

**Decision Variables:**

*  $x_1$: Litres of the first component used.
*  $x_2$: Litres of the second component used.

**Objective Function:**

* Maximize $Q = \ln(x_1 \cdot x_2) - \sqrt[3]{x_1 + 2x_2}$ 
    * This represents the quality of the end product as described in the problem.

**Constraints:**

1. **Tank Capacity:** $x_1 + x_2 \le 80$ 
    * The total volume of the two components cannot exceed the tank's capacity. 

2. **Stability Condition:** $(x_1 - \frac{x_1 + x_2}{2})^2 \le 10$ and $(x_2 - \frac{x_1 + x_2}{2})^2 \le 10$
    * The squared difference between the litres of each component and the mean litres used must be below 10 for stability.

3. **Non-negativity:** $x_1 \ge 0$, $x_2 \ge 0$
    *  The volume of each component cannot be negative.

**Complete Model:**

Maximize  $Q = \ln(x_1 \cdot x_2) - \sqrt[3]{x_1 + 2x_2}$

Subject to:

*  $x_1 + x_2 \le 80$
*  $(x_1 - \frac{x_1 + x_2}{2})^2 \le 10$ 
*  $(x_2 - \frac{x_1 + x_2}{2})^2 \le 10$
*  $x_1 \ge 0$
*  $x_2 \ge 0$ 


## 3. Generate the pyomo code

In [58]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [59]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
from pyomo.environ import *

# Create a model
model = ConcreteModel()

# Define decision variables
model.x1 = Var(domain=NonNegativeReals)
model.x2 = Var(domain=NonNegativeReals)

# Define the objective function
model.Q = Objective(expr=log(model.x1 * model.x2) - (model.x1 + 2*model.x2)**(1/3), sense=maximize)

# Define the constraints
model.tank_capacity = Constraint(expr=model.x1 + model.x2 <= 80)
model.stability1 = Constraint(expr=(model.x1 - (model.x1 + model.x2)/2)**2 <= 10)
model.stability2 = Constraint(expr=(model.x2 - (model.x1 + model.x2)/2)**2 <= 10)

# Solve the model
solver = SolverFactory('ipopt')  # You can choose a different solver if you prefer
results = solver.solve(model)

# Print the results
print(results)
print("x1 =", model.x1.value)
print("x2 =", model.x2.value)
print("Q =", model.Q.value) 
```

This code defines the optimization problem you provided using Pyomo. It first declares the decision variables, then defines the objective function and constraints. Finally, it uses the IPOPT solver to find the optimal solution. 

Note that this code does not include any sample data as the problem statement did not specify any parameters. You can easily modify this code to include parameter values if needed. 


## 4. Input problem data and try running the generated code

In [80]:
from pyomo.environ import *

# Create a model
model = ConcreteModel()

# Define decision variables
model.x1 = Var(domain=NonNegativeReals)
model.x2 = Var(domain=NonNegativeReals)

# Define the objective function
model.Q = Objective(expr=log(model.x1 * model.x2) - (model.x1 + 2*model.x2)**(1/3), sense=maximize)

# Define the constraints
model.tank_capacity = Constraint(expr=model.x1 + model.x2 <= 80)
model.stability1 = Constraint(expr=(model.x1 - (model.x1 + model.x2)/2)**2 <= 10)
model.stability2 = Constraint(expr=(model.x2 - (model.x1 + model.x2)/2)**2 <= 10)

# Solve the model
solver = SolverFactory('ipopt')  # You can choose a different solver if you prefer
results = solver.solve(model)

# Print the results
print(results)
print("x1 =", model.x1.value)
print("x2 =", model.x2.value)
print("Q =", model.Q.value) 


Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 3
  Number of variables: 2
  Sense: unknown
Solver: 
- Status: ok
  Message: Ipopt 3.11.1\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.03982973098754883
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

x1 = 43.162277233647174
x2 = 36.8377229631074


AttributeError: 'ScalarObjective' object has no attribute 'value'

## 5. Correct the code to verify model viability (optional)

## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [81]:
print(response.text)

## Mathematical Optimization Model 

**Parameters:**

*  None specified in the problem statement. 

**Decision Variables:**

*  $x_1$: Litres of the first component used.
*  $x_2$: Litres of the second component used.

**Objective Function:**

* Maximize $Q = \ln(x_1 \cdot x_2) - \sqrt[3]{x_1 + 2x_2}$ 
    * This represents the quality of the end product as described in the problem.

**Constraints:**

1. **Tank Capacity:** $x_1 + x_2 \le 80$ 
    * The total volume of the two components cannot exceed the tank's capacity. 

2. **Stability Condition:** $(x_1 - \frac{x_1 + x_2}{2})^2 \le 10$ and $(x_2 - \frac{x_1 + x_2}{2})^2 \le 10$
    * The squared difference between the litres of each component and the mean litres used must be below 10 for stability.

3. **Non-negativity:** $x_1 \ge 0$, $x_2 \ge 0$
    *  The volume of each component cannot be negative.

**Complete Model:**

Maximize  $Q = \ln(x_1 \cdot x_2) - \sqrt[3]{x_1 + 2x_2}$

Subject to:

*  $x_1 + x_2 \le 80$
*  $(x_1 - \frac{

In [82]:
print(response2.text)

```python
from pyomo.environ import *

# Create a model
model = ConcreteModel()

# Define decision variables
model.x1 = Var(domain=NonNegativeReals)
model.x2 = Var(domain=NonNegativeReals)

# Define the objective function
model.Q = Objective(expr=log(model.x1 * model.x2) - (model.x1 + 2*model.x2)**(1/3), sense=maximize)

# Define the constraints
model.tank_capacity = Constraint(expr=model.x1 + model.x2 <= 80)
model.stability1 = Constraint(expr=(model.x1 - (model.x1 + model.x2)/2)**2 <= 10)
model.stability2 = Constraint(expr=(model.x2 - (model.x1 + model.x2)/2)**2 <= 10)

# Solve the model
solver = SolverFactory('ipopt')  # You can choose a different solver if you prefer
results = solver.solve(model)

# Print the results
print(results)
print("x1 =", model.x1.value)
print("x2 =", model.x2.value)
print("Q =", model.Q.value) 
```

This code defines the optimization problem you provided using Pyomo. It first declares the decision variables, then defines the objective function and constraint